In [1]:
## construct references
annotations = {'human': '/hive/groups/recon/projs/primates/primates_indel_corrected_bionano_cut/reference/gencode.v27.annotation.no_PAR.gp',
              'orangutan': '/hive/groups/recon/projs/primates/primates_indel_corrected_bionano_cut/consensus_gene_set/Susie_Orangutan.filtered.gp',
              'chimp': '/hive/groups/recon/projs/primates/primates_indel_corrected_bionano_cut/consensus_gene_set/Clint_Chimp.filtered.gp',
              'gorilla': '/hive/groups/recon/projs/primates/primates_indel_corrected_bionano_cut/consensus_gene_set/Susie_Gorilla.filtered.gp',
              'ponAbe2': '/hive/groups/recon/projs/primates/original_primates/redo_annotation_indel/consensus_gene_set/Orangutan.filtered.gp',
              'panTro4': '/hive/groups/recon/projs/primates/original_primates/redo_annotation_indel/consensus_gene_set/Chimp.filtered.gp',
              'gorGor4': '/hive/groups/recon/projs/primates/original_primates/redo_annotation_indel/consensus_gene_set/Gorilla.filtered.gp'}

fastas = {'human': '/hive/groups/recon/projs/primates/primates_indel_corrected_bionano_cut/genome_files/Human.fa',
         'orangutan': '/hive/groups/recon/projs/primates/primates_indel_corrected_bionano_cut/genome_files/Susie_Orangutan.fa',
         'chimp': '/hive/groups/recon/projs/primates/primates_indel_corrected_bionano_cut/genome_files/Clint_Chimp.fa',
         'gorilla': '/hive/groups/recon/projs/primates/primates_indel_corrected_bionano_cut/genome_files/Susie_Gorilla.fa',
         'ponAbe2': '/hive/groups/recon/projs/primates/original_primates/redo_annotation_indel/genome_files/Orangutan.fa',
         'panTro4': '/hive/groups/recon/projs/primates/original_primates/redo_annotation_indel/genome_files/Chimp.fa',
         'gorGor4': '/hive/groups/recon/projs/primates/original_primates/redo_annotation_indel/genome_files/Gorilla.fa'}
from tools.fileOps import *
base_cmd = 'kallisto index -i {0}.kallisto {1}\n'
with open('kallisto_cmds.txt', 'w') as outf:
    for g, a in annotations.iteritems():
        f = fastas[g]
        tx_fa = g + '.transcripts.fa'
        outf.write(base_cmd.format(g, tx_fa))
        with TemporaryFilePath() as bed:
            !genePredToBed {a} {bed}
            !fastaFromBed -fi {f} -fo {tx_fa} -bed {bed} -name -split -s

In [ ]:
# add references for ensembl
from glob import glob
with open('kallisto_cmds.txt', 'a') as outf:
    for ref in ['pantro_ensembl', 'gorgor3_ensembl', 'ponabe_ensembl']:
        gtf = glob(ref + '/*gtf')
        assert len(gtf) == 1
        gtf = gtf[0]
        tx_fa = ref + '.transcripts.fa'
        if ref == 'gorgor3_ensembl':
            f = 'gorgor3_ensembl/Gorilla_gorilla.gorGor3.1.dna_sm.toplevel.fa'
        elif ref == 'pantro_ensembl': 
            f = 'pantro_ensembl/Pan_troglodytes.CHIMP2.1.4.dna_sm.toplevel.fa'
        else:
            f = fastas['ponAbe2']
        with TemporaryFilePath() as bed:
            !gtfToGenePred {gtf} /dev/stdout | genePredToBed /dev/stdin {bed}
            !fastaFromBed -fi {f} -fo {tx_fa} -bed {bed} -name -split -s
        outf.write(base_cmd.format(ref, tx_fa))

In [4]:
%%bash
cmd="cd ${PWD} && para make -cpu=1 -ram=32g kallisto_cmds.txt"
ssh ku $cmd

Process is interrupted.


In [52]:
import os
rnaseq_dir = '/hive/groups/recon/projs/primates/more_rnaseq_data/iPSC-data'
base_cmd = 'kallisto quant -t 8 -i {0}.kallisto {1} {2} -o {0}_kallisto_output\n'
with open('kallisto_cmds.txt', 'w') as outf:
    for g in ['human', 'orangutan', 'chimp', 'gorilla']:
        fwd = os.path.join(rnaseq_dir, g + '.fwd.fq')
        rev = os.path.join(rnaseq_dir, g + '.rev.fq')
        assert os.path.exists(fwd)
        assert os.path.exists(rev)
        outf.write(base_cmd.format(g, fwd, rev))
    # add in the original genome CAT sets
    for g, ref in [['chimp', 'panTro4'], ['gorilla', 'gorGor4'], ['orangutan', 'ponAbe2']]:
        fwd = os.path.join(rnaseq_dir, g + '.fwd.fq')
        rev = os.path.join(rnaseq_dir, g + '.rev.fq')
        assert os.path.exists(fwd)
        assert os.path.exists(rev)
        outf.write(base_cmd.format(ref, fwd, rev))
    # add in the ensembl sets
    for g, ref in [['chimp', 'pantro_ensembl'], ['gorilla', 'gorgor3_ensembl'], ['orangutan', 'ponabe_ensembl']]:
        fwd = os.path.join(rnaseq_dir, g + '.fwd.fq')
        rev = os.path.join(rnaseq_dir, g + '.rev.fq')
        assert os.path.exists(fwd)
        assert os.path.exists(rev)
        outf.write(base_cmd.format(ref, fwd, rev))

In [6]:
%%bash
cmd="cd ${PWD} && para make -cpu=32 -ram=256g kallisto_cmds.txt"
ssh ku $cmd

Process is interrupted.


In [53]:
# add in a comparison of each species-specific to human
import os
rnaseq_dir = '/hive/groups/recon/projs/primates/more_rnaseq_data/iPSC-data'
base_cmd = 'kallisto quant -t 8 -i human.kallisto {1} {2} -o {0}_human_kallisto_output\n'
with open('kallisto_human_cmds.txt', 'w') as outf:
    for g in ['orangutan', 'chimp', 'gorilla']:
        fwd = os.path.join(rnaseq_dir, g + '.fwd.fq')
        rev = os.path.join(rnaseq_dir, g + '.rev.fq')
        assert os.path.exists(fwd)
        assert os.path.exists(rev)
        outf.write(base_cmd.format(g, fwd, rev))

In [ ]:
%%bash
cmd="cd ${PWD} && para make -cpu=32 -ram=256g kallisto_human_cmds.txt"
ssh ku $cmd

In [2]:
# load dfs
import pandas as pd

base_dir = '/hive/groups/recon/projs/primates/primates_indel_corrected_bionano_cut/consensus_gene_set/'
dfs = {'chimp': pd.read_csv(os.path.join(base_dir, 'Clint_Chimp.fixed.filtered.gp_info'), sep='\t'),
      'orangutan': pd.read_csv(os.path.join(base_dir, 'Susie_Orangutan.fixed.filtered.gp_info'), sep='\t'),
      'gorilla': pd.read_csv(os.path.join(base_dir, 'Susie_Gorilla.fixed.filtered.gp_info'), sep='\t')}

base_dir = '/hive/groups/recon/projs/primates/original_primates/redo_annotation_indel/consensus_gene_set/'
for g, og in [['panTro4', 'Chimp'], ['ponAbe2', 'Orangutan'], ['gorGor4', 'Gorilla']]:
    dfs[g] = pd.read_csv(os.path.join(base_dir, og + '.filtered.gp_info'), sep='\t')

from tools.sqlInterface import *
dfs['human'] = load_annotation('/hive/groups/recon/projs/primates/primates_indel_corrected_bionano_cut/databases/Human.db')
dfs['human'].columns = ['transcript_id', 'gene_id', 'transcript_name', 'gene_name', 'gene_biotype', 'transcript_biotype']

In [20]:
# load ensembl annotations
from tools.misc import *
from glob import glob
def load_map(gtf):
    ensembl_map = {}
    biotype_map = {}
    for l in open(gtf):
        l = l.rstrip().split('\t')
        x = parse_gtf_attr_line(l[-1])
        try:
            ensembl_map[x['transcript_id']] = x['gene_name']
        except KeyError:
            continue
        try:
            biotype_map[x['transcript_id']] = x['gene_biotype']
        except KeyError:
            continue
    ensembl_map = pd.DataFrame(ensembl_map.items(), columns = ['transcript_id', 'gene_name'])
    biotype_map = pd.DataFrame(biotype_map.items(), columns = ['transcript_id', 'gene_biotype'])
    m = ensembl_map.merge(biotype_map, on='transcript_id')
    return m

    
for ref in ['pantro_ensembl', 'gorgor3_ensembl', 'ponabe_ensembl']:
    gtf = glob(ref + '/*gtf')
    assert len(gtf) == 1
    gtf = gtf[0]
    df = load_map(gtf)
    dfs[ref] = df

In [54]:
# load expression
exp_dfs = {}
for g in dfs:
    df = pd.read_csv('{}_kallisto_output/abundance.tsv'.format(g), header=0, sep='\t')
    df['target_id'] = [x.split('(')[0] for x in df.target_id]
    exp_dfs[g] = df
for g in ['gorilla_human', 'orangutan_human', 'chimp_human']:
    df = pd.read_csv('{}_kallisto_output/abundance.tsv'.format(g), header=0, sep='\t')
    df['target_id'] = [x.split('(')[0] for x in df.target_id]
    exp_dfs[g] = df

In [75]:
# merge dfs
merged_dfs = {}
for g, exp_df in exp_dfs.iteritems():
    if g == 'human' or 'ensembl' in g:
        m = exp_df.merge(dfs[g][['transcript_id', 'gene_name', 'gene_biotype']].drop_duplicates(), left_on='target_id', right_on='transcript_id')
        m = m[m.gene_biotype == 'protein_coding']
        m = m[['gene_name', 'tpm']]
    elif g in ['gorilla_human', 'orangutan_human', 'chimp_human']:
        m = exp_df.merge(dfs['human'][['transcript_id', 'gene_name', 'gene_biotype']].drop_duplicates(), left_on='target_id', right_on='transcript_id')
        m = m[m.gene_biotype == 'protein_coding']
        m = m[['gene_name', 'tpm']]
    else:
        m = exp_df.merge(dfs[g][['transcript_id', 'source_gene_common_name', 'gene_biotype']].drop_duplicates(), left_on='target_id', right_on='transcript_id')
        m = m[m.gene_biotype == 'protein_coding']
        m = m[['source_gene_common_name', 'tpm']]
        m.columns = ['gene_name', 'tpm']
    m = m.groupby('gene_name').aggregate(sum)
    merged_dfs[g] = m.reset_index()
    print g, m.tpm.sum()

gorGor4 715365.918296
ponAbe2 912589.675316
human 570790.558337
gorilla_human 584843.975313
panTro4 546909.102316
chimp 508775.855775
ponabe_ensembl 661078.184878
gorilla 607275.57229
orangutan_human 905687.0035
orangutan 930358.716555
gorgor3_ensembl 100309.753879
pantro_ensembl 378186.189785
chimp_human 517688.186924


In [76]:
# fix names
merged_dfs['human'].columns = ['gene_name', 'Human (GENCODE V27)']
merged_dfs['chimp'].columns = ['gene_name', 'Clint/Chimpanzee (CAT)']
merged_dfs['gorilla'].columns = ['gene_name', 'Susie/Gorilla (CAT)']
merged_dfs['orangutan'].columns = ['gene_name', 'Susie/Orangutan (CAT)']
merged_dfs['pantro_ensembl'].columns = ['gene_name', 'panTro4 (Ensembl V90)']
merged_dfs['ponabe_ensembl'].columns = ['gene_name', 'ponAbe2 (Ensembl V90)']
merged_dfs['gorgor3_ensembl'].columns = ['gene_name', 'gorGor3 (Ensembl V90)']
merged_dfs['panTro4'].columns = ['gene_name', 'panTro4 (CAT)']
merged_dfs['ponAbe2'].columns = ['gene_name', 'ponAbe2 (CAT)']
merged_dfs['gorGor4'].columns = ['gene_name', 'gorGor4 (CAT)']
merged_dfs['chimp_human'].columns = ['gene_name', 'GRCh38 (GENCODE V27)']
merged_dfs['orangutan_human'].columns = ['gene_name', 'GRCh38 (GENCODE V27)']
merged_dfs['gorilla_human'].columns = ['gene_name', 'GRCh38 (GENCODE V27)']

In [77]:
# now we merge and group
combined_dfs = {}
combined_dfs['chimp'] = merged_dfs['human'].merge(
    merged_dfs['chimp'], how='outer', on='gene_name').merge(
    merged_dfs['panTro4'], how='outer', on='gene_name').merge(
    merged_dfs['pantro_ensembl'], how='outer', on='gene_name').merge(
    merged_dfs['chimp_human'], how='outer', on='gene_name')
combined_dfs['gorilla'] = merged_dfs['human'].merge(
    merged_dfs['gorilla'], how='outer', on='gene_name').merge(
    merged_dfs['gorGor4'], how='outer', on='gene_name').merge(
    merged_dfs['gorgor3_ensembl'], how='outer', on='gene_name').merge(
    merged_dfs['gorilla_human'], how='outer', on='gene_name')
combined_dfs['orangutan'] = merged_dfs['human'].merge(
    merged_dfs['orangutan'], how='outer', on='gene_name').merge(
    merged_dfs['ponAbe2'], how='outer', on='gene_name').merge(
    merged_dfs['ponabe_ensembl'], how='outer', on='gene_name').merge(
    merged_dfs['orangutan_human'], how='outer', on='gene_name')

melted_dfs = {}
for g, df in combined_dfs.iteritems():
    df = pd.melt(df, id_vars=['gene_name', 'Human (GENCODE V27)'])
    df.columns = ['gene_name', 'Human (GENCODE V27)', 'assembly/annotation', 'TPM']
    melted_dfs[g] = df.drop_duplicates()

In [78]:
def plot_fn(pdf, df, cols):
    palette = sns.color_palette()
    #palette = [palette[3], palette[2], palette[1]]
    palette = palette[1:]
    with open(pdf, 'w') as outf, PdfPages(outf) as pdf:
        g = sns.lmplot(data=df, x='Human (GENCODE V27)', y='TPM', col='assembly/annotation',
                      hue='assembly/annotation', fit_reg=False,
                      scatter_kws={'marker': '+', 'alpha': 0.5, 's': 3,  'rasterized': True},
                      col_order=cols,
                      palette=palette)
        x = np.linspace(0, 250)
        for col, ax in zip(*[cols, g.axes[0]]):
            tmp = df[df['assembly/annotation'] == col]
            tmp = tmp[(tmp['Human (GENCODE V27)'].notnull()) & (tmp['TPM'].notnull())]
            c, p = pearsonr(tmp['Human (GENCODE V27)'], tmp['TPM'])
            ax.set_title(ax.get_title() + '\nPearson r={:.2f} p={:.2f}'.format(c, p))
            ax.set_xlim(0, 250)
            ax.set_ylim(0, 250)
            ax.plot(x, x, 'r--', color='black', alpha=0.7, linewidth=1)
        multipage_close(pdf, False)
        

import seaborn as sns
from cat.plots import *
from scipy.stats import *
for g, df in melted_dfs.iteritems():
    pdf = g + '.kallisto.pdf'
    if g == 'chimp':
        cols = ['GRCh38 (GENCODE V27)', 'Clint/Chimpanzee (CAT)', 'panTro4 (CAT)', 'panTro4 (Ensembl V90)']
    elif g == 'gorilla':
        cols = ['GRCh38 (GENCODE V27)', 'Susie/Gorilla (CAT)', 'gorGor4 (CAT)', 'gorGor3 (Ensembl V90)']
    else:
        cols = ['GRCh38 (GENCODE V27)', 'Susie/Orangutan (CAT)', 'ponAbe2 (CAT)', 'ponAbe2 (Ensembl V90)']
    plot_fn(pdf, df, cols)

In [79]:
num_human = len(merged_dfs['human'][merged_dfs['human']['Human (GENCODE V27)'] > 0.1])
num_cat = len(merged_dfs['chimp'][merged_dfs['chimp']['Clint/Chimpanzee (CAT)'] > 0.1])
num_old_cat = len(merged_dfs['panTro4'][merged_dfs['panTro4']['panTro4 (CAT)'] > 0.1])
num_ensembl = len(merged_dfs['pantro_ensembl'][merged_dfs['pantro_ensembl']['panTro4 (Ensembl V90)'] > 0.1])
num_human_chimp = len(merged_dfs['chimp_human'][merged_dfs['chimp_human']['GRCh38 (GENCODE V27)'] > 0.1])

In [80]:
print num_cat, num_old_cat, num_ensembl, num_human, num_human_chimp

15277 15242 13463 15776 16098
